Looking for the exceptions and saving them to a pickle file

In [1]:
import pickle
from random import choice as random_choice
import os

Here I will investigate what is happening for each of the board states that failed the test test_minimax.Test_Case.test_deterministic_outcomes_parallel
In each of these cases, the minimax function returned a different score when using parallel vs without.

To investigate this i will have each worker print out the result 

In [2]:
from minimax_parallel import *
from pieces import *
from assorted import *

In [3]:
class Move_Engine_Prime_Experimental(Move_Engine_Prime):
    def parallel_minimax(self, board_state, depth, cache_allowed, cache_manager, *args, **kwargs):
        # print("Parallel minimax called")

        # assert isinstance(depth, int)

        # assert depth >= 2, "depth must be greater or equal to 2 to do parallelization"

        is_maximizer = self.color_maximizer_key.get(board_state.next_to_go)

        # print("started getting moves_and_child_sorted")
        # print("getting legal moves, presorted")

        if self.parallel:
            moves_sorted = list(self.generate_move_child_in_parallel(
                board_state=board_state,
                depth=depth,
                is_maximizer=is_maximizer,
                give_child=False
            ))
        else:
            moves_sorted = list(self.generate_move_child(
                board_state=board_state,
                depth=depth,
                is_maximizer=is_maximizer,
                give_child=False
            ))


        # print("finished getting moves_and_child_sorted")
        # print("moves_sorted: ")
        # print(moves_sorted)
        

        legal_move_sub_arrays = self.break_up_legal_moves_to_segments(
            legal_moves=moves_sorted,
            number_sub_arrays=self.workers
        )
        legal_move_sub_arrays = list(legal_move_sub_arrays)

        # print("legal moves sub arrays generated")

        # print("Printing legal moves sub arrays")
        # print(legal_move_sub_arrays)

        # print("Defining job for workers")
        # use these legal moves to do many simultaneous minimax operations
        job = Minimax_Sub_Job(
            # move_engine=self,
            move_engine=Move_Engine(
                cache_manager=cache_manager,
                cache_allowed=cache_allowed
            ),
            board_state=board_state,
            depth=depth,
            cache_allowed=cache_allowed,
            cache_manager=cache_manager,
            args=args,
            kwargs=kwargs
        )
        # job = Minimax_Sub_Job(*(None,)*7)
        
        # print(f"dill.pickles(job)    -->    {dill.pickles(job)}")

        # assert dill.pickles(job)
        # assert dill.pickles(legal_move_sub_arrays)

        # print(f"Using pool to complete jobs in parallel (pool size = {cores})")

        # print("parallel_minimax: Using multiprocessing pool")
        with multiprocessing.Pool(self.workers) as pool:
            minimax_sub_job_results = pool.map(
                func = job,
                iterable=legal_move_sub_arrays
            )

        # print("parallel_minimax: multiprocessing finished")

        best_move = None
        if is_maximizer:
            best_score = 0-(ARBITRARILY_LARGE_VALUE +1)
        else:
            best_score = ARBITRARILY_LARGE_VALUE +1

        # select best move
        for worker, result in enumerate(minimax_sub_job_results):
            # print(f"worker {worker}:   ")
            # print(f"legal moves sub array   -->   {legal_move_sub_arrays[worker]}")
            # print(f"result  -->  {result}")
            print(f"worker {worker}:   result  -->  {result}")
            score, move = result

            if (is_maximizer and score > best_score) or (not is_maximizer and score < best_score):
                best_score = score
                best_move = move


        # print("finished getting best outcome")

        return best_score, best_move

In [4]:
def minimax(board_state: Board_State, is_maximizer: bool, depth, alpha, beta, check_extra_depth=True):
    # sourcery skip: low-code-quality, remove-unnecessary-else, swap-if-else-branches
    # assume white is maximizer
    # when calling, if give appropriate max min arg
    
    # base case 
    # if over or depth==0 return static evaluation
    over, _ = board_state.is_game_over_for_next_to_go()
    if depth == 0 or over:
        # special recursive case 1
        # examine terminal nodes that are check to depth 2 (variable depth)

        # to avoid goose chaises, extra resources are allowed if check not already explored
        if board_state.color_in_check() and check_extra_depth and not over:
            # print(f"checking board state {hash(board_state)} at additional depth due to check")
            return minimax(
                board_state=board_state,
                is_maximizer=is_maximizer,
                depth=2,
                alpha=alpha,
                beta=beta,
                check_extra_depth=False
            )
        else:
            # static eval works for game over to
            return board_state.static_evaluation(), None, None

    # define variables used to return more that just score (move and child)
    best_child_game_state: Board_State | None = None
    best_move_vector: Vector | None = None

    # function yields move ordered by how favorable they are (low depth minimax approximation)
    def gen_ordered_child_game_states():
        # this function does a low depth minimax recursive call (special recursive case 2) to give a move a score
        def approx_score_move(move):
            child_game_state = board_state.make_move(*move)

            return minimax(
                board_state=child_game_state,
                depth=depth-2,
                is_maximizer=not is_maximizer,
                alpha=alpha,
                beta=beta,
                check_extra_depth=False
            )[0]
            # print(f"approx_score_move(move={move!r})  ->  {result!r}")

        # if depth is 1 or less just yield moves form legal moves
        if depth <= 1 :
            yield from board_state.generate_legal_moves()
        # else sort them
        else:
            # sort best to worse
            # sort ascending order if minimizer, descending if maximizer
            yield from sorted(
                board_state.generate_legal_moves(),
                key=approx_score_move,
                reverse=is_maximizer
            )




    
    if is_maximizer:
        # set max to -infinity
        maximum_evaluation = (-1)*ARBITRARILY_LARGE_VALUE

        # iterate through moves and resulting game states
        for position_vector, movement_vector in gen_ordered_child_game_states():
            child_game_state = board_state.make_move(from_position_vector=position_vector, movement_vector=movement_vector)

            # evaluate each one
            # general recursive case 1
            evaluation, _, _ = minimax(
                board_state=child_game_state,
                is_maximizer=not is_maximizer,
                depth=depth-1,
                alpha=alpha,
                beta=beta,
                check_extra_depth=check_extra_depth
            )

            # update alpha and max evaluation
            if evaluation > maximum_evaluation:
                maximum_evaluation = evaluation
                best_child_game_state = child_game_state
                best_move_vector = (position_vector, movement_vector)
                alpha = max(alpha, evaluation)

            # where possible, prune
            if beta <= alpha:
                # print("Pruning!")
                break
        # once out of loop, return result
        return maximum_evaluation, best_child_game_state, best_move_vector

    else:
        minimum_evaluation = ARBITRARILY_LARGE_VALUE

        for position_vector, movement_vector in gen_ordered_child_game_states():
            child_game_state = board_state.make_move(from_position_vector=position_vector, movement_vector=movement_vector)
            evaluation, _, _ = minimax(
                board_state=child_game_state,
                is_maximizer=not is_maximizer,
                depth=depth-1,
                alpha=alpha,
                beta=beta,
                check_extra_depth=check_extra_depth
            )

            if evaluation < minimum_evaluation:
                minimum_evaluation = evaluation
                best_child_game_state = child_game_state
                best_move_vector = (position_vector, movement_vector)
                beta = min(beta, evaluation)

            if beta <= alpha:
                # print("Pruning!")
                break
        return minimum_evaluation, best_child_game_state, best_move_vector

def old_minimax(board_state: Board_State):
    is_maximizer = (board_state.next_to_go == "W")
    evaluation, best_child_game_state, best_move_vector =  minimax(
        board_state=board_state,
        depth = 2,
        is_maximizer=is_maximizer,
        alpha=-(ARBITRARILY_LARGE_VALUE+1),
        beta=(ARBITRARILY_LARGE_VALUE+1),
        check_extra_depth=False
    )
    return evaluation, best_move_vector

In [5]:
# failures: tuple[Board_State] = (
#     Board_State(pieces_matrix=((Rook(color='B'), Knight(color='B'), Queen(color='B'), None, King(color='B'), Bishop(color='B'), Knight(color='B'), Rook(color='B')), (Pawn(color='B'), None, Pawn(color='B'), Pawn(color='B'), Pawn(color='B'), Pawn(color='B'), Pawn(color='B'), None), (Bishop(color='B'), Pawn(color='B'), None, None, None, None, None, Pawn(color='B')), (None, None, None, None, None, None, None, None), (None, None, None, None, Pawn(color='W'), None, None, None), (Knight(color='W'), None, None, Pawn(color='W'), None, None, Pawn(color='W'), None), (Pawn(color='W'), Pawn(color='W'), Pawn(color='W'), None, None, Pawn(color='W'), None, Pawn(color='W')), (Rook(color='W'), None, Bishop(color='W'), Queen(color='W'), King(color='W'), Bishop(color='W'), Knight(color='W'), Rook(color='W'))), next_to_go='W', pieces_matrix_frequency={3176908122044428475: 1, 8533016422639447231: 1, -3374068713027168943: 1, 4802452147577338524: 1, 5488430360279548104: 1, 5597970105781533617: 1, -7213234226304348399: 1, -9077996078510467647: 1, -3759214679196013610: 1, 4005144678476986825: 1}),
#     Board_State(pieces_matrix=((None, None, None, Queen(color='B'), King(color='B'), None, Knight(color='B'), Rook(color='B')), (None, None, None, Pawn(color='B'), Bishop(color='W'), Pawn(color='B'), None, Pawn(color='B')), (Rook(color='W'), Rook(color='B'), Pawn(color='B'), None, Queen(color='W'), None, None, Bishop(color='B')), (None, Pawn(color='B'), None, None, None, None, None, None), (None, None, Pawn(color='W'), None, None, None, None, None), (None, Pawn(color='W'), None, None, None, Pawn(color='W'), None, Pawn(color='W')), (None, Pawn(color='W'), Pawn(color='W'), None, Pawn(color='W'), None, None, None), (None, Knight(color='W'), None, None, King(color='W'), Bishop(color='W'), Knight(color='W'), Rook(color='W'))), next_to_go='B', pieces_matrix_frequency={2967728354355788397: 1, -4712360490418558356: 1, 5660096862017806515: 1, -6980855437444764459: 1, -6833768341905155015: 1, 3725258599609543420: 1, 1467218662868185962: 1, 7733863292022932532: 1, 8776309253123531586: 1, -3931871940880748879: 1, -1244102050607895258: 1, -3851154746560537152: 1, -127080697591225239: 1, 3313016230643679282: 1, -8820117665714300011: 1, 8597594453879051089: 1, 6279537613164314886: 1, 403748989955141750: 1, -8328411228975039706: 1, 7684585326907715631: 1, 4597104056866507462: 1, 5640889646077649384: 1, 566926181390318557: 1, -6800960990701529166: 1, 1992798781959909034: 1, -6369694519830231224: 1, -5198768698936783946: 1, 3394910854399870049: 1, 7117034202181423583: 1, 1548568029865811182: 1, -5206431403276450454: 1, 3483600124936464346: 1, 8132412428664420472: 1, -8527254861488712713: 1, 7118432878314408321: 1}),
#     Board_State(pieces_matrix=((Rook(color='B'), Knight(color='B'), None, None, None, None, None, Rook(color='B')), (Pawn(color='B'), Pawn(color='B'), Pawn(color='B'), Bishop(color='B'), None, Pawn(color='B'), None, King(color='B')), (None, Queen(color='B'), None, Pawn(color='B'), None, None, Pawn(color='B'), Bishop(color='B')), (Knight(color='W'), None, None, Pawn(color='W'), None, None, None, Knight(color='B')), (None, Pawn(color='W'), None, None, None, None, None, None), (Pawn(color='W'), None, None, None, None, Pawn(color='W'), Pawn(color='W'), Pawn(color='B')), (None, Queen(color='W'), None, None, Pawn(color='W'), None, Rook(color='W'), Pawn(color='W')), (Rook(color='W'), None, None, None, King(color='W'), Bishop(color='W'), Bishop(color='W'), None)), next_to_go='B', pieces_matrix_frequency={-479119288580050187: 1, 8149356103551736299: 1, -4300499530981753615: 1, 7684396437600993105: 1, -166060383632609265: 1, -1441057148028276691: 1, 6687864042642248274: 1, 8155926174957050341: 1, -2702663650393261558: 1, 4036257676651567533: 1, -6636284627380925194: 1, -7606036792992762387: 1, 7126441055799876315: 1, -639626771934218182: 1, -2667635390213674894: 1, 286977095736326209: 1, -4500606871013043643: 1, 4636686396888491874: 1, 5179994481152610072: 1, -5281686996517227302: 1, -8187607215359493613: 1, -1270433788382970711: 1, -3241788906384712353: 1, -2477182383315935574: 1, 2923278441719800220: 1, 171839867146041831: 1, 7483451764588391241: 1, 2751359363776743931: 1, -1465624400966975198: 1, -5995828407469360282: 1, 4138842008089730048: 1, 1484752949813736883: 1, -744234504220292855: 1, 2294711820447417287: 1, 7938495173181068244: 1, 4773182063005038865: 1, -4561879528959745330: 1, -5029040819635607299: 1, -5664817013439017837: 1, -1479327491662753971: 1, 4050387587607345395: 1, -2155989479295216240: 1, 3466306244117044479: 1}),

# )

In [6]:
# with_parallel_engine_method_a = Move_Engine_Prime_Experimental()
# with_parallel_engine_method_b = Move_Engine_Prime_Experimental()
# without_parallel_engine = Move_Engine_Prime_Experimental()


# for engine in (with_parallel_engine_method_a, with_parallel_engine_method_b, without_parallel_engine):
#     engine.cache_allowed = False
#     engine.depth = 2
#     engine.additional_depth = 0



# def always(*args, **kwargs): return True
# def never(*args, **kwargs): return False


# # with_parallel_engine_method_a.break_up_legal_moves_to_segments = Move_Engine_Prime().break_up_legal_moves_to_segments
# with_parallel_engine_method_b.break_up_legal_moves_to_segments = Parallel_Move_Engine().break_up_legal_moves_to_segments

# with_parallel_engine_method_a.should_use_parallel = always
# with_parallel_engine_method_b.should_use_parallel = always
# without_parallel_engine.should_use_parallel = never


In [7]:
# for failure in failures:
#     is_maximizer = (failure.next_to_go == "W")
#     print(f"is_maximizer   -->   {is_maximizer}")
#     print("With parallel a:")
#     print(with_parallel_engine_method_a(failure))
#     print()
#     print("With parallel b:")
#     print(with_parallel_engine_method_b(failure))
#     print()
#     print("Without parallel:")
#     print(without_parallel_engine(failure))
#     print()
#     print("with old v2.4 minimax")
#     print(old_minimax(failure))
#     print()


In [8]:
# vanilla_move_engine = Move_Engine(cache_allowed=False, additional_depth=0)
# for failure in failures:
#     print(f"Board state:  {hash(failure)}")
#     print("Minimax vanilla move engine")
#     print(vanilla_move_engine(failure, 2))
#     print("Old v2.4 minimax")
#     print(old_minimax(failure))


In [9]:
def random_board_state(moves: int):
    board_state = Board_State()
    while True:
        try:
            for _ in range(moves):
                legal_moves = list(board_state.generate_legal_moves())
                # print(legal_moves)
                assert len(legal_moves) > 0
                random_move = random_choice(legal_moves)
                # print(f"Making move:   {board_state.get_piece_at_vector(random_move[0])}  {random_move[0].to_square()} to {(random_move[0] + random_move[1]).to_square()}")
                board_state = board_state.make_move(*random_move)

                # assert not board_state.is_game_over_for_next_to_go()

        except AssertionError:
            # print("Game over, trying again")
            continue
        else:
            return board_state

In [10]:
vanilla_move_engine = Move_Engine(cache_allowed=False, additional_depth=0)
def vanilla_minimax(board_state):
    return vanilla_move_engine(board_state, 2)


In [11]:
file_path = os.path.join(os.getcwd(), "failures.pkl")

def save_failures(failures: dict):
    with open(file_path, "wb") as file:
        file.write(
            pickle.dumps(
                failures
            )   
        )

def load_failures():
    if not os.path.exists(file_path):
        return dict()
    with open(file_path, "rb") as file:
        return pickle.loads(
            file.read()
        )


In [12]:
failures: dict[int, Board_State] = load_failures()
print(", ".join(map(str, failures.keys())))


In [13]:
while True:
    try:
        moves = random_choice(range(10, 50))
        board_state = random_board_state(moves)
        board_state_hash = hash(board_state)

        vanilla_score, _ = vanilla_minimax(board_state)
        old_score, _ = old_minimax(board_state)

        if old_score != vanilla_score:
            print(f"found new failure with hash:   {board_state_hash}")
            failures[board_state_hash] = board_state
            save_failures(failures)
    except KeyboardInterrupt:
        break
print(", ".join(map(str, failures.keys())))

In [ ]:
# vanilla_move_engine = Move_Engine(cache_allowed=False, additional_depth=0)

# run with variation to move engine (print statements and no pruning)

for failure in failures.values():
    print(f"Board state:  {hash(failure)}")
    print("Minimax vanilla move engine")
    print(vanilla_move_engine(failure, 2))
    print("Old v2.4 minimax")
    print(old_minimax(failure))
